# Hybrid Recommendation using
a) Collaborative Filtering Model

b) Content-based Model

In [18]:
import pandas as pd
from random import randint
import numpy as np
from numpy.linalg import norm 
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

In [19]:
'''
FORMULA
X(NORMALIZED) = (X - Xminimum)/(Xmaximum - Xminimum)

 min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    
    return [x/max_val for x in data]
'''
#normalizing the input between 0s and 1s
def normalize(data):    
    return (data - np.min(data)/ (np.max(data)- np.min(data)))


In [20]:
''' 
CREATING FUNCTIONS FOR CONTENT BASED RECOMMENDATION MODELS

''' 

#performing one hot encoding
def encoding(df, column):
    new = pd.get_dummies(df[column])
    new.reset_index(drop = True, inplace = True)
    return pd.concat([df, new], axis = 1)

class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        This function will calculate the cosine similarity between two vectors
        '''
        return sum(np.dot(v1,v2)/(norm(v1)*norm(v2)))
    
    def recommend(self, book_id, n_rec):
        """
        df (dataframe): The dataframe
        song_id (string): Representing the song name
        n_rec (int): amount of rec user wants
        """
        
        # calculate similarity of input book_id vector w.r.t all other vectors
        inputVec = self.df.loc[book_id].values
        self.df['sim']= self.df.apply(lambda x: self.cosine_sim(inputVec, x.values), axis=1)

        # returns top n user specified books
        return self.df.nlargest(columns='sim',n=n_rec)

In [21]:
'''
CREATING FRUNCTION FOR COLLABORATIVE FILTER MODELLING
'''

def svd(mat, df, factors):
    if not 1<= factors < min(mat.shape):
        raise ValueError("Must be 1 <= factors < min(mat.shape)")
    
    #matrix factorization
    u,s,v = svds(mat, k = factors)
    s = np.diag(s)

    #calculating the prediction ratings
    pred = np.dot(np.dot(u,s),v)
    pred = normalize(pred) #normalizing the predictions

    new_df = pd.DataFrame(pred, columns = df.columns, index = list(df.index) ).transpose()

    return new_df


In [22]:
''' 
CREATING THE FINAL HYBRID MODEL



def hybrid(reader_id, book_id, data, n_recs, cosine, svd_model):

    # similarity values

    s = list(enumerate(cosine[int(book_id)]))
    s = sorted(s, key = lambda x:x[1], reverse = True)

    # metadeta
    index = [i[0] for i in s]
    books = data.iloc[index][['book_id', 'book_rating', 'num_pages', 'publish_year', 'book_price', 'reader_id']]

    #applying the model

    books['predicted'] = books.apply(lambda x: svd_model.predict(reader_id, x['book_id'], x['book_rating']).predicted, axis = 1)
    
    #sorting

    books = books.sort_values('predicted', ascending = False)

    return books.head(n_recs)

'''

" \nCREATING THE FINAL HYBRID MODEL\n\n\n\ndef hybrid(reader_id, book_id, data, n_recs, cosine, svd_model):\n\n    # similarity values\n\n    s = list(enumerate(cosine[int(book_id)]))\n    s = sorted(s, key = lambda x:x[1], reverse = True)\n\n    # metadeta\n    index = [i[0] for i in s]\n    books = data.iloc[index][['book_id', 'book_rating', 'num_pages', 'publish_year', 'book_price', 'reader_id']]\n\n    #applying the model\n\n    books['predicted'] = books.apply(lambda x: svd_model.predict(reader_id, x['book_id'], x['book_rating']).predicted, axis = 1)\n    \n    #sorting\n\n    books = books.sort_values('predicted', ascending = False)\n\n    return books.head(n_recs)\n\n"

In [31]:
def generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000):
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d

In [32]:
df = generate_data(dataset_size = 100000)
df.to_csv('data.csv', index = False)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,1890,139,6,26723,115,9,16,2021,47,1
1,2833,75,6,10063,528,2,7,2004,88,2
2,2839,53,5,3218,385,2,39,2021,113,5
3,665,334,10,12375,515,8,4,2014,101,5
4,1649,43,9,20516,432,3,3,2003,188,6


In [33]:
df = df.sort_values(by=['book_id'], ascending = True)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
48962,1,53,2,27727,639,2,48,2002,148,3
74619,1,221,8,13915,491,8,35,2019,13,7
20803,1,215,6,19513,174,7,13,2008,95,4
20894,1,347,3,24807,313,10,18,2005,46,1
75231,1,297,2,21639,203,8,17,2004,105,1


In [34]:
if __name__ == '__main__':
    
    df = df

    # normalizing
    df['num_pages_norm'] = normalize(df['num_pages'].values)
    df['book_rating_norm'] = normalize(df['book_rating'].values)
    df['book_price_norm'] = normalize(df['book_price'].values)
    
    # One hot encoding
    df = encoding(df = df, column = 'publish_year')
    df = encoding(df = df, column = 'book_genre')
    df = encoding(df = df, column = 'text_lang')


    # generate a pivot table
    pivot = df.pivot_table(
        columns = 'book_id',
        index = 'reader_id',
        values = 'book_rating'
    ).fillna(0)

    # convert to a csr matrix
    mat = pivot.values
    mat = csr_matrix(mat)
    
    #applying SVD model
    pred_df = svd(mat, pivot, 10)
    
    ''' 
    # drop redundant columns
    cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
    df.drop(columns = cols, inplace = True)
    df.set_index('book_id', inplace = True)

    '''

    
    # ran on a sample as an example
    t = pred_df.copy()
    cbr = CBRecommend(df = t)
    print(cbr.recommend(book_id = t.index[0], n_rec = 5))


TypeError: 'numpy.float64' object is not iterable